# Daily Challenge: Pokemon Win Prediction Analysis

What you need to do:

1. Data Preparation
   - Load and merge pokemon.csv and combats.csv.
   - Fix missing values:
   - Fill the missing Name for Pokemon #62 (Primeape).
   - Handle NaN values in Type 2 (mark as “None” if missing).
   - Calculate each Pokemon’s win percentage using the combat data.
  

2. Exploratory Analysis & Visualization
    - Create a correlation matrix to identify relationships between stats (HP, Attack, Speed) and win percentage.
    - Plot a Seaborn pairplot or PairGrid for stats vs. win percentage.
    - Analyze the top 10 Pokemon by win percentage and their stats.
    

3. Machine Learning
    - Split data into training/testing sets (80/20 split).
    - Train and evaluate 3 regression models (e.g., Linear Regression, Random Forest, XGBoost) to predict win percentage.
    - Compare model performance using Mean Absolute Error (MAE).




### Data Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sns.set_theme()

In [2]:
pokemon = pd.read_csv("pokemon.csv")
combats = pd.read_csv("combats.csv")

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [4]:
combats.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [11]:
pokemon.loc[62, 'Name']="Primeape"

In [17]:
pokemon = pokemon["Type 2"].fillna("None")

In [24]:
wins = (combats["Winner"].value_counts())

In [29]:
first_counts = combats['First_pokemon'].value_counts()
second_counts = combats['Second_pokemon'].value_counts()

total_games = first_counts.add(second_counts, fill_value=0)

win_percentage = (wins / total_games) * 100
win_percentage = win_percentage.sort_values(ascending=False)
win_percentage

155    98.449612
513    97.478992
704    96.800000
20     96.638655
154    96.453901
         ...    
237     3.252033
639     3.100775
190     2.459016
290     2.173913
231          NaN
Length: 784, dtype: float64